# Sentinel 2
### Eleonora (Nori) Parelius
### 20.10.2025

# Sentinel 2
* go to [wiki page on Sentinel 2 at Copernicus](https://sentiwiki.copernicus.eu/web/s2-mission)
* skim for 5 minutes and try to identify 3-5 key characteristics of Sentinel 2
* optionally, identify one question you think the text can answer

# Sentinel 2
* Part of European Space Agency's (ESA) Copernicus programme
* Twin sattelites
    * Sentinel 2A since 23. June 2015
    * Sentinel 2B since 7. March 2017
    * Sentinel 2C, launched on 5.9.2024 -- to replace 2A
    
<img src="images/sentinelsat.jpg" alt="Image of Sentinel 2 from Copernicus" width="400"/>

## Coverage and time resolution
* At same angle every 5 days (10 days for same sattelite)
* Between $56^{\circ}$ S and $84^{\circ}$ degrees N
* Coast up to 20 km from land
* Islands over $100\ \rm{km^2}$
* Antarctica on demand
* Sun synchronous orbit at 786 km, 10:30 am descending node
<img src="images/sentinelcoverage.png" alt="Coverage of Sentinel 2" width="400"/>  
Image from Copernicus

## Sentinel 2 multispectral sensor
* 13 spectral bands in VIS, VNIR, SWIR
* Spatial resolution 10, 20, 60 m
* Push-broom, swath of 290 km
![](images/sentinelbands.jpg)

Image from eoportal.org


## Sentinel tiles
<img src="images/S2_granules_tiles.jpg" alt="Sentinel 2 tiles vs granules" width="300"/>

Image from eox.at 

## Products
All free!

* Level-1B
    * Top-of-atmosphere calibrated radiances in sensor geometry
    * Include radiometric corrections
    * Not orthorectified
    * Come in granules
* Level-1C
    * Top-of-atmosphere reflectance
    * Radiometrically and geometrically corrected
    * Orthorectified and spatially coregistered on a global reference system. 
    * Come in tiles
* Level-2A
    * Bottom-of-atmosphere (Surface) reflectance
    * Otherwise same as L-1C

## Examples

Let's look at the [Copernicus browser](https://browser.dataspace.copernicus.eu/)


# Sentinel 2 - Searching, downloading, loading up and working with it in Python

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Area of interest

First half of 2022, there was a drought in Norway, so we could expect that the vegetation in 2022 wasn't as healthy as usual. This should show up by NDVI being generally lower in 2022 than in 2021. 

### Pick your area
I am picking my home, you can pick wherever you want. 

### Go to Copernicus Browser, define area and download
* [Copernicus browser](https://browser.dataspace.copernicus.eu/)
* Use the *Search* tab and find one image from May 2021 and one from May 2022
* Download the tile

## Check the files
Where are the images? Unzip and explore the folder structure.

## Let's load it

In [ ]:
# Rewrite these to fit your images
may_2021_r10 = 'data/S2B_MSIL2A_20210530T104619_N0500_R051_T32VPM_20230228T112003.SAFE/GRANULE/L2A_T32VPM_A022099_20210530T104621/IMG_DATA/R10m/'
may_2022_r10 = 'data/S2A_MSIL2A_20220517T103631_N0400_R008_T32VPM_20220517T161111.SAFE/GRANULE/L2A_T32VPM_A036041_20220517T103708/IMG_DATA/R10m/'

In [ ]:
import rasterio

# Open b2, b3, b4 and b8
# Rewrite these to fit your images
# 2021
m2021_band2=rasterio.open(may_2021_r10 + "T32VPM_20210530T104619_B02_10m.jp2")
m2021_band3=rasterio.open(may_2021_r10 + "T32VPM_20210530T104619_B03_10m.jp2")
m2021_band4=rasterio.open(may_2021_r10 + "T32VPM_20210530T104619_B04_10m.jp2")
m2021_band8=rasterio.open(may_2021_r10 + "T32VPM_20210530T104619_B08_10m.jp2")

2022
m2022_band2=rasterio.open(may_2022_r10 + "T32VPM_20220517T103631_B02_10m.jp2")
m2022_band3=rasterio.open(may_2022_r10 + "T32VPM_20220517T103631_B03_10m.jp2")
m2022_band4=rasterio.open(may_2022_r10 + "T32VPM_20220517T103631_B04_10m.jp2")
m2022_band8=rasterio.open(may_2022_r10 + "T32VPM_20220517T103631_B08_10m.jp2")

# Check the properties of one of the bands
print(m2021_band2.profile)
# You can check the others too

## Plotting the bands

In [ ]:
# Fill in the blanks
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(m2022_band4.read(1), cmap='Reds')
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(..., cmap='Greens')
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(..., cmap='Blues')

## Plotting the RGB
That comes with the granule

In [ ]:
from rasterio.plot import show
RGB_im = rasterio.open('data/S2A_MSIL2A_20220517T103631_N0400_R008_T32VPM_20220517T161111.SAFE/S2A_MSIL2A_20220517T103631_N0400_R008_T32VPM_20220517T161111-ql.jpg')
# What do you get when you look at the properties?

In [ ]:
print(RGB_im.profile)

In [ ]:
show(RGB_im)

## Limiting the area

### Define the polygon

In [ ]:
import shapely
import pyproj
from shapely.ops import transform

# bounding box coordinates of the area of interest in 
# WGS84 CRS, [minx, miny, maxx, maxy]
bbox_coords = [60, 11.35, 60.21, 11.8] # find your

# Let's create a shapely geometry of the bounding box
polygon = shapely.geometry.box(*bbox_coords) 
# what happens if you remove the asterisk?

# Get it into the right coordinate system
project = pyproj.Transformer.from_proj(
    pyproj.Proj('epsg:4326'), # source coordinate system
    pyproj.Proj('epsg:32632')) # destination coordinate system 
# ! make sure you change to your destination coordinate system!

transformed_polygon = transform(project.transform, polygon)  # apply projection

### Crop all the bands and save as one tif

In [ ]:
import rasterio.mask
out_band2, out_transform = rasterio.mask.mask(m2021_band2, 
                                              [transformed_polygon], 
                                              crop=True)
out_meta = m2021_band2.meta

out_meta.update({"driver": "GTiff",
                 "count" : 4,
                 "height": out_band2.shape[1],
                 "width": out_band2.shape[2],
                 "transform": out_transform})

out_band3,_ = rasterio.mask.mask(m2021_band3, [transformed_polygon], crop=True)
out_band4,_ = rasterio.mask.mask(m2021_band4, [transformed_polygon], crop=True)
out_band8,_ = rasterio.mask.mask(m2021_band8, [transformed_polygon], crop=True)

with rasterio.open("may2021_r10_masked.tif", "w", **out_meta) as dest:
    dest.write(out_band2.squeeze(),1)
    dest.write(out_band3.squeeze(),2)
    dest.write(out_band4.squeeze(),3)
    dest.write(out_band8.squeeze(),4)

Do the same for 2022

## Let's plot it as RGB

In [ ]:
img1 = rasterio.open("may2021_r10_masked.tif")
rgb_image1 = np.array([img1.read(3), img1.read(2), img1.read(1)])
# Values in optical bands come in digital units DN, 
# where DN = 10 000 * reflectance where reflectance is 0-1
# Note, there is an offset of 1000 that is added to DNs 
# to avoid negative numbers.
# https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/
# Using 3 to brighten the images

print(np.unique(rgb_image1))

rgb_image1=(rgb_image1-1000.0)/10000*3
rgb_image1[rgb_image1>1]=1
rgb_image1[rgb_image1<0]=0

img2 = rasterio.open("may2022_r10_masked.tif")
rgb_image2 = np.array([img2.read(3), img2.read(2), img2.read(1)])
rgb_image2=(rgb_image2-1000.0)/10000*3
rgb_image2[rgb_image2>1]=1
rgb_image2[rgb_image2<0]=0

# Use print statements to find out what kind of values the image comes as 

In [ ]:
# Plot the resulting images
fig, ax = plt.subplots(1,2, figsize=(20,12))
show(rgb_image1, ax=ax[0])
show(rgb_image2, ax=ax[1])

## NDVI images

In [ ]:
# Fill in the gaps ...
# There is an offset of 1000 to the values
red_2021 = (img1.read(...) - ...).astype(int)
red_2021[red_2021<0] = 0
red_2022 = (img2.read(...) - ...).astype(int)
red_2022[red_2022<0] = 0

nir_2021 = (img1.read(...) - ...).astype(int)
nir_2021[nir_2021<0] = 0
nir_2022 = (img2.read(...) - ...).astype(int)
nir_2022[nir_2022<0] = 0


In [ ]:
# Then we can calculate the NDVI
# Fill in the gaps
ndvi_2021 = (...)/(... + 1e-6)
ndvi_2022 = (...)/(... + 1e-6)

# Are you getting values that make sense?

In [ ]:
# Mean NDVI
print("Mean NDVI in 2021:", ndvi_2021.mean())
print("Mean NDVI in 2022:", ndvi_2022.mean())

In [ ]:
# And let's plot the NDVI in grayscale
# Higher value gives darker color
fig, ax = plt.subplots(1,2, figsize=(12,5))
ax[0].imshow(ndvi_2021, cmap='Grays')
ax[1].imshow(ndvi_2022, cmap='Grays')

## Let's look at the difference

We expect the health of the vegetation to be better in 2021, because of the drought in 2022. The NDVI should then be higher for 2021 than for 2022.

We should then expect the difference ndvi_2021 - ndvi_2022 to be mostly positive. 

Let's check if that is the case. 

In [ ]:
# Plotting the NDVI difference

diff = ndvi_2021 - ndvi_2022
fig, ax = plt.subplots(1, figsize=(6,6))
pic = ax.imshow(diff, norm=matplotlib.colors.CenteredNorm(),cmap = cm.coolwarm)
fig.colorbar(pic, ax=ax)

In [ ]:
diff_binary = diff
diff_binary[diff_binary>0]=1
diff_binary[diff_binary<0]=-1
fig, ax = plt.subplots(1, figsize=(6,6))
pic = ax.imshow(diff_binary, norm=matplotlib.colors.CenteredNorm(),cmap = cm.coolwarm)

## Save the NDVI difference as a Geotiff

In [ ]:
# Open one of the orginal geotiffs, so we can copy the metadata from it
with rasterio.open('may2021_r10_masked.tif') as src:
    out_image = diff
    out_meta = src.meta.copy()
    # Update the metadata that are different for our new image
    out_meta.update({
        "driver": "GTiff",
        "count": 1, #only one band instead of 12
        "dtype": "float64", #float instead of int
    })
# Use rasterio to write the numpy array into a geotiff 
# using the defined metadata
with rasterio.open("NDVI_diff.tiff", "w", **out_meta) as dest:
    dest.write(diff, indexes=1)